# Serotonin 3D GNN Project


This project builds upon research done by Łapińska et al. (2024): https://doi.org/10.3390/pharmaceutics16030349

Data used: https://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/releases/chembl_35/

Move the unpacked chembl_35_sqlite.tar.gz file into the data/ dir.

The research linked above presents two Quantitative Structure-Activity Relationship (QSAR) models to predict serotonergic binding affinity and selectivity, respectively, using Mordred molecular 2D descriptors. Specifically, one model classifies compounds binarily as "active" or "inactive", with a cutoff of pKi = 7. Another model does multiclass classification to predict the serotonergic selectivity of compounds previously classified as "active".

I am following a similar approach, but using 3D molecular graph representations instead of 2D molecular descriptors as input modality and using only the ChEMBL database, not ZINC.


## Google Colab Setup


### Configuration


In [1]:
from pathlib import Path

IN_COLAB = False

PATH_NOTEBOOK = (
    Path("/content/drive/MyDrive/Colab Notebooks/serotonin-3d-gnn.ipynb")
    if IN_COLAB
    else Path(
        "/Users/paul/Library/CloudStorage/GoogleDrive-unoutsch@gmail.com/My Drive/Colab Notebooks/serotonin-3d-gnn.ipynb"
    )
)
PATH_REPO = (
    Path("/content/drive/MyDrive/Repositories/serotonin-3d-gnn")
    if IN_COLAB
    else Path.cwd()
)
PATH_DATA = PATH_REPO / "data"

### Syncing Google Drive with Google Colab Content


In [2]:
if IN_COLAB:
    from google.colab import drive

    drive.mount("/content/drive")

### Installing Requirements


In [3]:
%pip install -r "$PATH_REPO/requirements.txt"

Note: you may need to restart the kernel to use updated packages.


## Imports


In [47]:
import os
import pandas as pd
import pickle
from rdkit import Chem
from rdkit.Chem import AllChem
import shutil
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

## Utils


### Syncing this file between Colab and local Git repo


Make sure the paths exist.


In [7]:
if IN_COLAB:
    shutil.copyfile(PATH_NOTEBOOK, PATH_REPO / "serotonin-3d-gnn.ipynb")
else:
    shutil.copyfile(PATH_REPO / "serotonin-3d-gnn.ipynb", PATH_NOTEBOOK)

## Data


### Note on Data Aquisition from chembl_35.db

In order to collect the desired data from the ChEMBL SQL database and transform it into a .csv file, I undertook the steps detailed in `data/README.md`.


### Loading the Data


If the pickled torch_data_list already exists, load it.


In [ ]:
pickle_file_path = PATH_DATA / "torch_data_list.pkl"

if os.exists(pickle_file_path):
    torch_data_list = pickle.load(open(pickle_file_path, "rb"))
    print("Loaded torch_data_list from pickle file")
else:
    print("Creating torch_data_list from scratch")

#### Load dataframe from .csv file


In [ ]:
# if df is None:
df = pd.read_csv(PATH_DATA / "serotonin_binding_summary.csv")

# remove columns that have less than 1000 non-NaN values
# df = df.dropna(axis=1, thresh=1000)

df.describe()

,molecule_id,Serotonin (5-HT) receptor,Serotonin 1 (5-HT1) receptor,Serotonin 1 receptors; 5-HT1B & 5-HT1D,Serotonin 1a (5-HT1a) receptor,Serotonin 1b (5-HT1b) receptor,Serotonin 1d (5-HT1d) receptor,Serotonin 1e (5-HT1e) receptor,Serotonin 1f (5-HT1f) receptor,Serotonin 2 (5-HT2) receptor,...,Serotonin 2b (5-HT2b) receptor,Serotonin 2c (5-HT2c) receptor,Serotonin 3 (5-HT3) receptor,Serotonin 3a (5-HT3a) receptor,Serotonin 3b (5-HT3b) receptor,Serotonin 4 (5-HT4) receptor,Serotonin 5a (5-HT5a) receptor,Serotonin 5b (5-HT5b) receptor,Serotonin 6 (5-HT6) receptor,Serotonin 7 (5-HT7) receptor
count,2.345600e+04,90.000000,252.000000,1.0,9462.000000,1492.000000,1472.000000,91.000000,127.000000,1469.000000,...,2337.000000,4343.000000,939.000000,1040.000000,8.000000,1009.000000,422.000000,1.00,4221.000000,3100.000000
mean,1.003325e+06,6.081759,6.683902,6.2,7.258523,6.952528,7.554968,5.791172,7.458423,7.053201,...,6.603829,6.810210,7.625768,7.047520,7.203500,7.645809,6.573801,7.17,7.311171,6.977487
std,8.986583e+05,0.926906,1.104283,NaN,1.152004,1.226482,1.365880,0.652239,0.859240,1.159567,...,0.981462,1.032874,1.225942,1.535413,1.735342,1.179482,1.089819,NaN,1.143388,1.016128
min,9.700000e+01,4.390000,4.100000,6.2,4.000000,4.000000,4.000000,4.800000,5.140000,4.030000,...,4.190000,4.000000,4.010000,4.000000,5.460000,5.000000,4.070000,7.17,4.120000,4.000000
25%,2.291570e+05,5.385000,5.800000,6.2,6.480000,6.050000,6.470000,5.360000,7.000000,6.240000,...,5.900000,6.050000,6.800000,5.700000,5.490000,6.810000,5.800000,7.17,6.470000,6.285000
50%,5.757615e+05,5.925000,6.600000,6.2,7.280000,6.850000,7.640000,5.730000,7.720000,6.920000,...,6.523333,6.740000,7.700000,7.185000,7.015000,7.640000,6.390000,7.17,7.360000,6.990000
75%,1.965967e+06,6.657500,7.585000,6.2,8.060000,7.850000,8.700000,6.175000,8.020000,8.000000,...,7.210000,7.512500,8.580000,8.410000,8.393500,8.400000,7.068750,7.17,8.110000,7.700000
max,2.881244e+06,9.800000,9.300000,6.2,11.000000,10.000000,10.700000,8.200000,8.800000,10.300000,...,10.100000,10.700000,10.420000,10.400000,9.604000,10.800000,9.170000,7.17,10.400000,10.000000


In [ ]:
df_targets = df.drop(columns=["molecule_id", "canonical_smiles"])
print(f"Number of targets: {len(df_targets.columns)}")

Number of targets: 21


In [ ]:
def create_torch_data(smiles: str, targets: torch.Tensor) -> Data:
    # getting RDKit molecule object
    mol = Chem.MolFromSmiles(smiles)

    if mol is None:
        return None

    # add explicit hydrogen atoms to the molecule (are not included in the SMILES string) so that its 3D structure is complete
    mol = Chem.AddHs(mol)

    # EmbedMolecule positions atoms of mol in 3D space stochastically; if it fails (returning -1) return None
    if AllChem.EmbedMolecule(mol, randomSeed=42) == -1:
        return None

    # optimize the 3D structure using Universal Force Field (UFF) to lower mol's energy
    AllChem.UFFOptimizeMolecule(mol)

    # conformer contains 3D coordinates for mol's atoms
    conformer = mol.GetConformer()

    # atom-level features and 3D positions
    atom_features, positions = [], []
    for atom in mol.GetAtoms():
        features = [
            atom.GetAtomicNum(),  # atomic number: uid of element (e.g., 6 for carbon, 8 for oxygen)
            atom.GetDegree(),  # degree: number of bonds connecting the atom
            atom.GetFormalCharge(),  # formal charge: atom's electrical charge
            int(
                atom.GetHybridization()
            ),  # hybridization: type of atom's orbital hybridization (e.g., sp, sp2) as int
        ]
        atom_features.append(features)

        # 3D coordinates of atom from conformer
        pos = conformer.GetAtomPosition(atom.GetIdx())
        positions.append([pos.x, pos.y, pos.z])

    # transform to PyTorch tensors
    x = torch.tensor(atom_features, dtype=torch.float)
    pos = torch.tensor(positions, dtype=torch.float)

    # bonds between atoms – indices of connected atoms as well as types and conjugation
    edge_index, edge_attr = [], []
    for bond in mol.GetBonds():
        # indices of bonded atoms
        i, j = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()

        bond_feat = [
            bond.GetBondTypeAsDouble(),  # bond type as float (e.g., 1.0 for single, 2.0 for double bonds)
            (
                1.0 if bond.GetIsConjugated() else 0.0
            ),  # conjugation flag indicating whether the bond is conjugated (1.0 if true, else 0.0) (conjugated means that electrons are delocalized, moving freely between multiple atoms)
        ]
        # for undirected graph, add bond in both directions
        edge_index += [[i, j], [j, i]]
        edge_attr += [bond_feat, bond_feat]

    # transform to PyTorch tensors
    # edge_index tensor is transposed to fit torch_geometric's expected shape (2, number_of_edges).
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)

    # graph as PyTorch Geometric Data object
    # x: atom features, [atomic number, degree, formal charge, hybridization]
    # pos: 3D positions of atoms, [x, y, z]
    # edge_index: connectivity indices between atoms, [[i, j], [j, i]]
    # edge_attr: features per bond, [[bond type, conjugation], [bond type, conjugation]]
    return Data(x=x, pos=pos, edge_index=edge_index, edge_attr=edge_attr, y=targets)

In [44]:
torch_data_list = [
    create_torch_data(
        row["canonical_smiles"],
        torch.tensor(df_targets.loc[row.name].values, dtype=torch.float),
    )
    for _, row in df.iterrows()
]

[11:17:01] UFFTYPER: Unrecognized charge state for atom: 1
[11:17:01] UFFTYPER: Unrecognized charge state for atom: 1
[11:17:10] UFFTYPER: Unrecognized charge state for atom: 14
[11:17:10] UFFTYPER: Unrecognized charge state for atom: 14
[11:17:50] UFFTYPER: Unrecognized charge state for atom: 12
[11:17:50] UFFTYPER: Unrecognized charge state for atom: 12
[11:20:02] UFFTYPER: Unrecognized charge state for atom: 21
[11:20:02] UFFTYPER: Unrecognized charge state for atom: 21
[11:21:17] UFFTYPER: Unrecognized charge state for atom: 12
[11:21:17] UFFTYPER: Unrecognized charge state for atom: 12
[11:21:28] UFFTYPER: Unrecognized charge state for atom: 1
[11:21:28] UFFTYPER: Unrecognized charge state for atom: 1
[11:21:34] UFFTYPER: Warning: hybridization set to SP3 for atom 29
[11:21:34] UFFTYPER: Warning: hybridization set to SP3 for atom 29
[11:21:34] UFFTYPER: Warning: hybridization set to SP3 for atom 29
[11:21:36] UFFTYPER: Warning: hybridization set to SP3 for atom 27
[11:21:36] UFFTY

In [ ]:
pickle_file_path = PATH_DATA / "torch_data_list.pkl"

with open(pickle_file_path, "wb") as f:
    pickle.dump(torch_data_list, f)

print(f"Saved torch_data_list to {pickle_file_path}")

Saved torch_data_list to /Users/paul/My Drive/Repositories/serotonin-3d-gnn/data/torch_data_list.pkl


Create training and test sets.


In [81]:
filtered_torch_data_list = [d for d in torch_data_list if d is not None]
print(
    f"Number of items in filtered_torch_data_list: {len(filtered_torch_data_list)} / {len(torch_data_list)}"
)  # still retaining original torch_data_list for reference to df later

split_idx = int(0.8 * len(filtered_torch_data_list))

data_graph_train = DataLoader(
    filtered_torch_data_list[:split_idx], batch_size=32, shuffle=True
)
data_graph_test = DataLoader(
    filtered_torch_data_list[split_idx:], batch_size=32, shuffle=False
)

print(
    f"# training graphs: {len(data_graph_train)}\n# test graphs: {len(data_graph_test)}"
)
print(data_graph_train.dataset[0])

Number of items in filtered_torch_data_list: 23439 / 23456
# training graphs: 586
# test graphs: 147
Data(x=[49, 4], edge_index=[2, 104], edge_attr=[104, 2], y=[21], pos=[49, 3])


## Models


### Model 1: PyTorch Implementation of a 3D GCN

In this section, a 3D graph convolutional network is created using PyTorch. The model takes as input a 3D molecular graph and outputs predictions of the serotonergic binding affinity of the molecule.

Information about the graph input the model will receive and process:

-   The feature matrix H contains the node (atom) features. Each row corresponds to a node, and each column corresponds to a feature.
-   The adjacency matrix A is built from the edge_index tensor, which contains the indices of the edges in the graph. The matrix A is built under the hood of the GCNConv class.


#### Model Architecture


In [82]:
from torch_geometric.nn import GCNConv, global_mean_pool
from torch.nn import Linear
import torch.nn.functional as F


class SeroGCN(torch.nn.Module):
    def __init__(self, n_in, n_hidden, n_out):
        super(SeroGCN, self).__init__()

        self.conv1 = GCNConv(n_in, n_hidden)
        self.conv2 = GCNConv(n_hidden, n_hidden)

        self.fc = Linear(n_hidden, n_out)

    def forward(self, mol: Data):
        x, pos, edge_index, edge_attr = mol.x, mol.pos, mol.edge_index, mol.edge_attr

        x = self.conv1(x, edge_index)  # TODO: include pos and edge_attr in computation
        x = F.relu(x)

        x = self.conv2(x, edge_index)
        x = F.relu(x)

        x = global_mean_pool(
            x, mol.batch
        )  # global mean pooling aggregates node features, returning a single graph-level vectorial representation

        x = self.fc(x)

        return x

### Model 2: Pretrained 3D GNN (...)


## Training


In [83]:
if torch.cuda.is_available():
    print("Using CUDA")
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    print("Using MPS")
    device = torch.device("mps")
else:
    print("Using CPU")
    device = torch.device("cpu")

Using MPS


In [84]:
epochs = 10
n_in = data_graph_train.dataset[0].num_features
n_hidden = 32
n_out = len(df_targets.columns)
print(f"Node features: {n_in}, number of outputs: {n_out}")

Node features: 4, number of outputs: 21


In [88]:
def masked_mse_loss(pred, target):
    # mask of valid (non-nan) targets
    mask = ~torch.isnan(target)
    if mask.sum() == 0:
        # return 0 loss, so that it doesn't affect the gradient
        return torch.tensor(0.0, requires_grad=True, device=target.device)
    # squared error for entries that are valid
    loss = (pred[mask] - target[mask]) ** 2
    return loss.mean()

In [ ]:
def train(
    model: torch.nn.Module, data_loader: DataLoader, optimizer: torch.optim.Optimizer
):
    model.train()
    for epoch in range(epochs):
        for data in data_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data)
            # masked loss to ignore NaNs
            loss = masked_mse_loss(out, data.y.view(-1, 21))
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch+1}: Loss = {loss.item()}")

In [92]:
sero_gcn = SeroGCN(n_in=n_in, n_hidden=n_hidden, n_out=n_out).to(device)
sero_gcn_optimizer = torch.optim.Adam(sero_gcn.parameters(), lr=0.01)
# sero_gcn_criterion = torch.nn.MSELoss()

In [93]:
train(sero_gcn, data_graph_train, sero_gcn_optimizer)

Epoch 0: Loss = 1.2581521272659302
Epoch 1: Loss = 1.5593031644821167
Epoch 2: Loss = 1.6708483695983887
Epoch 3: Loss = 1.4416097402572632
Epoch 4: Loss = 1.5349111557006836


KeyboardInterrupt: 